In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn.ensemble as ml
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [57]:
#Save the new files in a folder
save_name = "RF_Imputed_TACC_Files"
os.makedirs(save_name, exist_ok = True)

save_path = save_name + "/"

In [21]:
file_1 = "TO_TACC_STM_STG.xlsx"
file_2 = "TO_TACC_RCOMPL.xlsx"

In [22]:
df_1 = pd.read_excel(file_1)
df_2 = pd.read_excel(file_2)

## Random Forest For File #1

In [23]:
#LABEL ENCODING CATEGORICAL VALUES
#all of the quantitative values for file 1
quant_df = df_1.loc[:, df_1.dtypes == float]
quant_df2 = df_1.loc[:, df_1.dtypes == int]
totalquant_df = pd.concat([quant_df, quant_df2], axis = 1, sort = False)
totalquant_df.head()

,FRAC_GRADIENT,BREAKDOWN_PRESSURE,HYDROSTATIC_PRESSURE,TREAT_AVG_PRESSURE,TREAT_MAX_PRESSURE,TREAT_MIN_PRESSURE,PROPPANT_DESIGNED,PROPPANT_IN_FORMATION,PROPPANT_IN_WELLBORE,AVG_HYDRAULIC_POWER,...,PUMPSONLINENO,USERNUM1,USERNUM3,DURPUMP,TOP_DEPTH_TVD,BOTTOM_DEPTH_TVD,PROPOSED,Stage_Length (meters),Prod_Per_Stage,Errors_Per_Stage
0,1.0,6547.0,4984.0,9919.0,10176.0,6550.0,104000.0,104060.0,0.0,16793.0,...,NaN,NaN,NaN,NaN,11508.0,11504.0,0.0,65.0,36707.875000,9.300000
1,1.0,7348.0,5031.0,10027.0,10525.0,7348.0,104000.0,104000.0,0.0,22069.0,...,NaN,NaN,NaN,NaN,11598.0,11604.0,0.0,65.0,35123.342105,9.078947
2,1.0,6740.0,5576.0,9358.0,9765.0,7346.0,152125.0,152125.0,NaN,12780.0,...,16.0,18.0,13.0,120.0,12872.0,12876.0,0.0,73.0,3218.482759,7.810345
3,1.0,5100.0,5724.0,7280.0,9347.0,5100.0,0.0,0.0,0.0,8618.0,...,NaN,NaN,NaN,NaN,12991.0,13064.0,0.0,5264.0,0.000000,3.630435
4,1.0,6026.0,4696.0,8298.0,9349.0,6026.0,104000.0,104780.0,0.0,13950.0,...,NaN,NaN,NaN,NaN,10829.0,10826.0,0.0,65.0,23077.516129,10.000000


In [24]:
#all of the qualitative values for file 1
qual_df = df_1.loc[:, df_1.dtypes == object]
qual_df2 = df_1.loc[:, df_1.dtypes == 'datetime64[ns]']
clmn2 = list(qual_df2)
for i in clmn2:
    qual_df2[i] = qual_df2[i].astype(object)
totalqual_df = pd.concat([qual_df, qual_df2], axis = 1, sort = False)
totalqual_df.head()

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,DELIVERY_MODE,BOTTOM_HOLE_PRESSURE_METHOD,STIMULATION_RESULT,STIMULATION_RESULT_DETAIL,STIMTREAT_ID,FLUID_NAME,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,STG_START_DATE,STG_END_DATE
0,WELL - 2019000507,Producing,COMPLETIONS,04/10/2017 00:00:00,04/26/2018 09:30:00,COMPLETED,OCM - PP PUC,INITIAL COMPLETION,04/14/2017 17:15:00,08/19/2017 18:00:00,...,Casing,NaN,Success,According to Plan,6652C6217F11405488E785A3D84CFD56,25# XL,2017-07-26 06:00:00,2017-08-18 18:00:00,2017-07-27 02:31:00,2017-07-27 04:31:00
1,WELL - 2019000477,Producing,COMPLETIONS,12/06/2017 07:00:00,08/06/2018 18:00:00,NaN,OCM - PP PUC,INITIAL COMPLETION,12/11/2017 17:00:00,02/09/2018 09:00:00,...,Casing,Calculated,Success,According to Plan,9808C6BE9B2746A9A63D58EEB3885939,25# GUAR BORATE,2018-01-20 06:00:00,2018-09-02 09:00:00,2018-01-23 20:33:00,2018-01-23 21:22:00
2,WELL - 2019000699,Producing,COMPLETIONS,12/16/2018 06:00:00,10/12/2019 10:00:00,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,01/11/2019 06:00:00,02/06/2019 07:00:00,...,Casing,NaN,Success,According to Plan,A6066AE1E85F4B1DB15D9940B5FBBEBF,Slickwater,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 08:05:00,2019-01-28 10:05:00
3,WELL - 2019000665,Producing,COMPLETIONS,01/21/2018 10:00:00,10/09/2018 07:00:00,COMPLETED,OCM - PP TUB,RECOMPLETION,01/27/2018 06:00:00,02/23/2018 18:00:00,...,Casing,NaN,Success,According to Plan,6ACEA3AEC54E415B8251B86897A229F3,FRESH WATER,2018-02-14 08:00:00,2018-02-23 18:00:00,2018-02-19 14:40:00,2018-02-20 00:19:00
4,WELL - 2019000466,Producing,COMPLETIONS,08/01/2017 06:00:00,NaN,PRODUCING,OCM - PP PUC,INITIAL COMPLETION,08/25/2017 06:00:00,10/05/2017 06:00:00,...,Casing,NaN,Success,According to Plan,0459426A0021476CADD000E29A193E8D,Slick Water,2017-08-09 06:00:00,2017-02-10 06:00:00,2017-11-09 13:35:00,2017-11-09 15:33:00


In [25]:
#use label encoding to change qualitative values for file 1
original = totalqual_df
mask = df_1.isnull()
totalqual_df = totalqual_df.astype(str).apply(LabelEncoder().fit_transform)
totalqual_df = totalqual_df.where(~mask, original)

In [26]:
#concatenate qualitative and quantitative values together
result = pd.concat([totalqual_df, totalquant_df], axis = 1, sort = False)
result.head()

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,PUMPSONLINENO,USERNUM1,USERNUM3,DURPUMP,TOP_DEPTH_TVD,BOTTOM_DEPTH_TVD,PROPOSED,Stage_Length (meters),Prod_Per_Stage,Errors_Per_Stage
0,182,1,0,56,63,0,2,0,88,159,...,NaN,NaN,NaN,NaN,11508.0,11504.0,0.0,65.0,36707.875000,9.300000
1,153,1,0,168,96,NaN,2,0,235,26,...,NaN,NaN,NaN,NaN,11598.0,11604.0,0.0,65.0,35123.342105,9.078947
2,217,1,0,175,133,0,1,1,7,25,...,16.0,18.0,13.0,120.0,12872.0,12876.0,0.0,73.0,3218.482759,7.810345
3,209,1,0,12,128,0,3,1,24,32,...,NaN,NaN,NaN,NaN,12991.0,13064.0,0.0,5264.0,0.000000,3.630435
4,142,1,0,119,NaN,2,2,0,172,192,...,NaN,NaN,NaN,NaN,10829.0,10826.0,0.0,65.0,23077.516129,10.000000


In [27]:
#change -1 to null, add back to original file
df_replaced_values = result.where(df_1 != -1, np.nan)

In [28]:
#columns that have NaNs
missing_df = df_replaced_values.loc[:, df_replaced_values.isna().any()]

In [29]:
for i in range(0, len(df_replaced_values.columns)):
    print(df_replaced_values.columns[i])

#WELL_NAME
CURRENT_STATUS
JOB_CATEGORY
JOB_START_DATE
JOB_END_DATE
JOB_STATUS
LIKE_KIND
PROJECT_TYPE
PHASE_START_DATE
PHASE_END_DATE
PHASE
JOB_OP_CODE
RIG_ID
RIG_TYPE
CONTRACTOR
REPORT_START_DATE
REPORT_END_DATE
STIMULATION_TREAT_DATE
STIMULATION_TREAT_TYPE
STIM_TREAT_COMPANY
STIM_TREAT_SUPERVISOR
DIVERSION_COMPANY
DIVERSION_METHOD
DELIVERY_MODE
BOTTOM_HOLE_PRESSURE_METHOD
STIMULATION_RESULT
STIMULATION_RESULT_DETAIL
STIMTREAT_ID
FLUID_NAME
RIG_ACCEPT_DATE
RIG_RELEASE_DATE
STG_START_DATE
STG_END_DATE
FRAC_GRADIENT
BREAKDOWN_PRESSURE
HYDROSTATIC_PRESSURE
TREAT_AVG_PRESSURE
TREAT_MAX_PRESSURE
TREAT_MIN_PRESSURE
PROPPANT_DESIGNED
PROPPANT_IN_FORMATION
PROPPANT_IN_WELLBORE
AVG_HYDRAULIC_POWER
MAX_PUMP_POWER
MAX_PUMP_RATING
TREAT_AVG_RATE
TREAT_MAX_RATE
TREAT_MIN_RATE
POST_INSTANT_SHUT_IN_PRESSURE
INITIAL_SHUT_IN_PRESSURE
PRE_INSTANT_SHUT_IN_PRESSURE
STG_TOP_DEPTH
STG_BOTTOM_DEPTH
STG_ELEMENT_TOP_DEPTH
STG_ELEMENT_BOTTOM_DEPTH
STG_INITIAL_PUMPING_PRESSURE
STG_FINAL_PUMPING_PRESSURE
STG_BREA

In [33]:
independentVariables = ['STIMTREAT_ID', 'REPORT_START_DATE_YEAR', 'UWI', 'UWI_SIDETRACK', 'STG_NUMBER', 'CURRENT_STATUS', 'JOB_CATEGORY', 'STG_SLURRY_VOLUME_PUMPED', 'STG_CLEAN_VOLUME_PUMPED', 'STIMULATION_TREAT_TYPE', 'CONTRACTOR']

for i in independentVariables:
    if not i in df_replaced_values:
        print(i, "False")

REPORT_START_DATE_YEAR False
UWI False
UWI_SIDETRACK False
STG_NUMBER False
STG_SLURRY_VOLUME_PUMPED False
STG_CLEAN_VOLUME_PUMPED False


In [30]:
#machine learning
missing_var_clmns = list(missing_df.columns)

independentVariables = ['STIMTREAT_ID', 'REPORT_START_DATE_YEAR', 'UWI', 'UWI_SIDETRACK', 'STG_NUMBER', 'CURRENT_STATUS', 'JOB_CATEGORY', 'STG_SLURRY_VOLUME_PUMPED', 'STG_CLEAN_VOLUME_PUMPED', 'STIMULATION_TREAT_TYPE', 'CONTRACTOR']
for i in missing_var_clmns:
    #creating 2 datasets
    dataWithValues = df_replaced_values[pd.isnull(df_replaced_values[i]) == False]
    dataWithoutValues = df_replaced_values[pd.isnull(df_replaced_values[i])]
    
    #training the model
    rfModel_variables = RandomForestRegressor()
    rfModel_variables.fit(dataWithValues[independentVariables], dataWithValues[i])
   
    #predicting the values
    generatedVariables = rfModel_variables.predict(X = dataWithoutValues[independentVariables])
    
    #append back to original dataset
    dataWithoutValues[i] = generatedVariables.astype(int)
    data = dataWithValues.append(dataWithoutValues)
    df_replaced_values = data

KeyError: "['STG_SLURRY_VOLUME_PUMPED', 'UWI', 'REPORT_START_DATE_YEAR', 'STG_CLEAN_VOLUME_PUMPED', 'UWI_SIDETRACK', 'STG_NUMBER'] not in index"

In [68]:
#finished imputation
data.head()

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,Prod_Per_Stage,Errors_Per_Stage,UWI,UWI_SIDETRACK,REPORT_START_DATE_YEAR,STG_NUMBER,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,Rig_Duration (days),Stg_Duration (hrs)
8146,118,2,0,108,53,0,2,0,142,169,...,7670.857143,8.0,2019000441,0,2018,38,9925,10483,32.0,1.0
8444,118,2,0,108,53,0,2,0,142,169,...,7670.857143,8.0,2019000441,0,2018,33,10075,10641,32.0,2.0
9834,118,2,0,108,53,0,2,0,142,169,...,7670.857143,8.0,2019000441,0,2018,37,10010,10570,32.0,2.0
11398,118,2,0,108,53,0,2,0,142,169,...,7670.857143,8.0,2019000441,0,2018,32,10018,10572,32.0,2.0
12391,118,2,0,108,53,0,2,0,142,169,...,7670.857143,8.0,2019000441,0,2018,35,10105,10670,32.0,2.0


In [69]:
#making sure that there are no more null values
data.isnull().any(axis=0)

#WELL_NAME                       False
CURRENT_STATUS                   False
JOB_CATEGORY                     False
JOB_START_DATE                   False
JOB_END_DATE                     False
JOB_STATUS                       False
LIKE_KIND                        False
PROJECT_TYPE                     False
PHASE_START_DATE                 False
PHASE_END_DATE                   False
PHASE                            False
JOB_OP_CODE                      False
RIG_ID                           False
RIG_TYPE                         False
CONTRACTOR                       False
REPORT_START_DATE                False
REPORT_END_DATE                  False
STIMULATION_TREAT_DATE           False
STIMULATION_TREAT_TYPE           False
STIM_TREAT_COMPANY               False
STIM_TREAT_SUPERVISOR            False
DIVERSION_COMPANY                False
DIVERSION_METHOD                 False
DELIVERY_MODE                    False
BOTTOM_HOLE_PRESSURE_METHOD      False
STIMULATION_RESULT       

In [70]:
data.to_excel(save_path + "RF_imputed_" + file_1,index=False)

## Random Forest For File #2

In [71]:
#LABEL ENCODING CATEGORICAL VALUES
#all of the quantitative values for file 2
quant_df = df_2.loc[:, df_2.dtypes == float]
quant_df2 = df_2.loc[:, df_2.dtypes == int]
totalquant_df = pd.concat([quant_df, quant_df2], axis = 1, sort = False)
totalquant_df.head()

,FRAC_GRADIENT,BREAKDOWN_PRESSURE,HYDROSTATIC_PRESSURE,TREAT_AVG_PRESSURE,TREAT_MAX_PRESSURE,TREAT_MIN_PRESSURE,PROPPANT_DESIGNED,PROPPANT_IN_WELLBORE,AVG_HYDRAULIC_POWER,MAX_PUMP_POWER,...,PROPPANT_IN_FORMATION,STG_NUMBER,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,BOTTOM_DEPTH_TVD,Rig_Duration (days),Stg_Duration (hrs),Stage_Length (meters)
0,1.0,6740.0,5576.0,9358.0,9765.0,7346.0,152125.0,NaN,12780.0,13932.0,...,152125,1,17968,18041,4466,4632,12876,34,2,73
1,1.0,5100.0,5724.0,7280.0,9347.0,5100.0,0.0,0.0,8618.0,12279.0,...,0,1,13063,18327,24452,24452,13064,9,9,5264
2,1.0,4901.0,5087.0,4999.0,5384.0,NaN,0.0,0.0,7864.0,NaN,...,1,2,12103,16589,5043,5043,11646,4,2,4486
3,1.0,5304.0,5623.0,9496.0,9602.0,NaN,130000.0,NaN,11606.0,16880.0,...,130160,1,18290,18355,2923,3015,12988,5,1,65
4,1.0,7340.0,5494.0,9523.0,9916.0,NaN,366938.0,NaN,16441.0,19362.0,...,366100,27,13694,13842,7337,7677,12676,9,2,148


In [72]:
#all of the qualitative values for file 2
qual_df = df_2.loc[:, df_2.dtypes == object]
qual_df2 = df_2.loc[:, df_2.dtypes == 'datetime64[ns]']
clmn2 = list(qual_df2)
for i in clmn2:
    qual_df2[i] = qual_df2[i].astype(object)
totalqual_df = pd.concat([qual_df, qual_df2], axis = 1, sort = False)
totalqual_df.head()

/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,CLOSURE_PRESSURE_METHOD,STIMULATION_RESULT,STIMULATION_RESULT_DETAIL,STIMTREAT_ID,FLUID_NAME,SITE_SUPERVISOR,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,STG_START_DATE,STG_END_DATE
0,WELL - 2019000699,Producing,COMPLETIONS,12/16/2018 06:00:00,10/12/2019 10:00:00,COMPLETED,OCM - MECHANICAL ISOLATION,RECOMPLETION,01/11/2019 06:00:00,02/06/2019 07:00:00,...,NaN,Success,According to Plan,A6066AE1E85F4B1DB15D9940B5FBBEBF,Slickwater,MIKE HOOSE,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 08:05:00,2019-01-28 10:05:00
1,WELL - 2019000665,Producing,COMPLETIONS,01/21/2018 10:00:00,10/09/2018 07:00:00,COMPLETED,OCM - PP TUB,RECOMPLETION,01/27/2018 06:00:00,02/23/2018 18:00:00,...,NaN,Success,According to Plan,6ACEA3AEC54E415B8251B86897A229F3,FRESH WATER,NaN,2018-02-14 08:00:00,2018-02-23 18:00:00,2018-02-19 14:40:00,2018-02-20 00:19:00
2,WELL - 2019000347,Producing,COMPLETIONS,08/31/2017 13:00:00,NaN,NaN,OCM - PP PUC,RECOMPLETION,09/07/2017 00:00:00,10/29/2017 17:00:00,...,Nolte-Smith Calc,Success,According to Plan,29626CB5494D45ACB9C7723B028E9DEF,Slick Water,NaN,2017-10-25 06:00:00,2017-10-29 06:00:00,2017-10-26 10:13:00,2017-10-26 12:45:00
3,WELL - 2019001412,Producing,COMPLETIONS,02/13/2019 07:30:00,NaN,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,03/04/2019 16:30:00,04/14/2019 16:00:00,...,NaN,Success,According to Plan,DCCB82735EEA4540A1C0C402180C5A45,FRP-1S,CHRIS SANCHEZ,2019-03-24 18:00:00,2019-03-30 06:00:00,2019-03-25 14:05:00,2019-03-25 15:34:00
4,WELL - 2019001390,Producing,COMPLETIONS,04/04/2019 07:30:00,06/15/2019 18:00:00,NaN,OCM - MECHANICAL ISOLATION,RECOMPLETION,04/23/2019 07:00:00,06/15/2019 18:00:00,...,NaN,Success,According to Plan,B8733A01BFFD46CC8CCDE726BD4B484B,FRP-1S,JW KRAMER,2019-06-06 06:00:00,2019-06-15 06:00:00,2019-06-13 07:17:00,2019-06-13 09:19:00


In [73]:
#use label encoding to change qualitative values for file 1
original = totalqual_df
mask = df_2.isnull()
totalqual_df = totalqual_df.astype(str).apply(LabelEncoder().fit_transform)
totalqual_df = totalqual_df.where(~mask, original)

In [74]:
#concatenate qualitative and quantitative values together
result = pd.concat([totalqual_df, totalquant_df], axis = 1, sort = False)
result.head()

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,PROPPANT_IN_FORMATION,STG_NUMBER,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,BOTTOM_DEPTH_TVD,Rig_Duration (days),Stg_Duration (hrs),Stage_Length (meters)
0,29,0,0,65,19,0,1,0,1,6,...,152125,1,17968,18041,4466,4632,12876,34,2,73
1,21,0,0,2,18,0,3,0,2,8,...,0,1,13063,18327,24452,24452,13064,9,9,5264
2,5,0,0,48,NaN,NaN,2,0,47,49,...,1,2,12103,16589,5043,5043,11646,4,2,4486
3,65,0,0,6,NaN,NaN,1,0,10,17,...,130160,1,18290,18355,2923,3015,12988,5,1,65
4,62,0,0,18,9,NaN,1,0,19,27,...,366100,27,13694,13842,7337,7677,12676,9,2,148


In [75]:
#change -1 to null, add back to original file
df_replaced_values = result.where(df_2 != -1, np.nan)

In [76]:
#columns that have NaNs
missing_df = df_replaced_values.loc[:, df_replaced_values.isna().any()]

In [77]:
#machine learning
missing_var_clmns = list(missing_df.columns)

independentVariables = ['STIMTREAT_ID', 'REPORT_START_DATE_YEAR', 'UWI', 'UWI_SIDETRACK', 'STG_NUMBER', 'CURRENT_STATUS', 'JOB_CATEGORY', 'STG_SLURRY_VOLUME_PUMPED', 'STG_CLEAN_VOLUME_PUMPED', 'STIMULATION_TREAT_TYPE', 'CONTRACTOR']
for i in missing_var_clmns:
    #creating 2 datasets
    dataWithValues = df_replaced_values[pd.isnull(df_replaced_values[i]) == False]
    dataWithoutValues = df_replaced_values[pd.isnull(df_replaced_values[i])]
    
    #training the model
    rfModel_variables = RandomForestRegressor()
    rfModel_variables.fit(dataWithValues[independentVariables], dataWithValues[i])
   
    #predicting the values
    generatedVariables = rfModel_variables.predict(X = dataWithoutValues[independentVariables])
    
    #append back to original dataset
    dataWithoutValues[i] = generatedVariables.astype(int)
    data = dataWithValues.append(dataWithoutValues)
    df_replaced_values = data

/Users/kellychiu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

/Users/kellychiu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kellychiu/anaconda3/lib/python3.7/site-package

/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kellychiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kellychiu/anaconda3/lib/python3.7/site-package

In [78]:
#finished imputation
data.head()

,#WELL_NAME,CURRENT_STATUS,JOB_CATEGORY,JOB_START_DATE,JOB_END_DATE,JOB_STATUS,LIKE_KIND,PROJECT_TYPE,PHASE_START_DATE,PHASE_END_DATE,...,PROPPANT_IN_FORMATION,STG_NUMBER,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,STG_CLEAN_VOLUME_PUMPED,STG_SLURRY_VOLUME_PUMPED,BOTTOM_DEPTH_TVD,Rig_Duration (days),Stg_Duration (hrs),Stage_Length (meters)
366,45,0,0,15,23,0,2,0,15,22,...,0,1,13600,16654,21820,21820,13172,120.0,9.0,3054
672,35,0,0,11,17,0,1,0,14,23,...,162000,1,17223,17342,3780,3953,12758,7.0,1.0,119
695,35,0,0,11,15,0,1,0,14,23,...,366938,2,17055,17203,6907,7525,12753,7.0,2.0,148
715,35,0,0,11,19,0,1,0,14,23,...,360822,3,16887,17034,7038,7431,12749,7.0,1.0,147
732,35,0,0,11,12,0,1,0,14,23,...,366938,4,16719,16867,7343,7742,12744,7.0,1.0,148


In [79]:
#making sure that there are no more null values
data.isnull().any(axis=0)

#WELL_NAME                       False
CURRENT_STATUS                   False
JOB_CATEGORY                     False
JOB_START_DATE                   False
JOB_END_DATE                     False
JOB_STATUS                       False
LIKE_KIND                        False
PROJECT_TYPE                     False
PHASE_START_DATE                 False
PHASE_END_DATE                   False
PHASE                            False
JOB_OP_CODE                      False
RIG_ID                           False
RIG_TYPE                         False
CONTRACTOR                       False
REPORT_START_DATE                False
REPORT_END_DATE                  False
STIMULATION_TREAT_DATE           False
STIMULATION_TREAT_TYPE           False
STIM_TREAT_COMPANY               False
STIM_TREAT_SUPERVISOR            False
DIVERSION_COMPANY                False
DIVERSION_METHOD                 False
DELIVERY_MODE                    False
BOTTOM_HOLE_PRESSURE_METHOD      False
CLOSURE_PRESSURE_METHOD  

In [80]:
data.to_excel(save_path + "RF_imputed_" + file_2,index=False)